# Straighten an image using the Hough transform

We'll write our own Hough transform to compute the Hough transform and use it to straighten a wonky image.

## Compiler parameters

Set my Jypyter environment for the use of OpenCV in a C++ notebook. You don't need this line when you write your own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use CMake.

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <stdexcept>
#include <sstream>
#include <string>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Read the image from a file on the disk and return a new matrix

![../wonky.png](../wonky.png)

In [5]:
Mat image = imread("../wonky.png");

## Check for errors

In [6]:
// Check for failure
if (image.empty()) 
{
    throw runtime_error("Could not open or find the image");
}

## Convert degrees to gradients

In [7]:
double deg2rad(double anAngleInDegrees)
{
    return anAngleInDegrees * M_PI / 180.0;   
}

## Apply the Canny edge detector

In [8]:
Mat cannyEdgeDetector(const Mat& anInputImage,
                   int aCannyThreshold)
{
    // Find edges using Canny
    Mat edge_image;
    const int ratio = 3;
    const int kernel_size = 3;
    
    Canny(anInputImage,
          edge_image,
          aCannyThreshold,
          aCannyThreshold * ratio,
          kernel_size);
    
    return edge_image;
}

In [9]:
Mat blurred_image;
blur( image, blurred_image, Size(3,3) );
Mat edge = cannyEdgeDetector(blurred_image, 60);

namedWindow("edge", WINDOW_GUI_EXPANDED);
imshow("edge", edge);

namedWindow("image", WINDOW_GUI_EXPANDED);
imshow("image", image);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

| Original image | canny |
|----------------|--------|
|![image](../wonky_original.png) |  ![canny](../wonky_canny.png) |

## Compute the accumulator

In [10]:
Mat houghTransform(const Mat& anInputImage,
                   int aCannyThreshold)
{
    // Blur the input image
    Mat blurred_image;
    blur( anInputImage, blurred_image, Size(3,3) );

    // Find edges using Canny
    Mat edge_image = cannyEdgeDetector(blurred_image, aCannyThreshold);

    int width  = 180;
    double diagonal = sqrt(edge_image.cols * edge_image.cols + edge_image.rows * edge_image.rows);
    int height = floor(2.0 * diagonal);
    double half_height = height / 2.0;
    Mat accumulator(height, width, CV_32F, Scalar(0));

    // Process all the pixels of the edge image
    for (int j = 0; j < edge_image.rows; ++j)
    {
        for (int i = 0; i < edge_image.cols; ++i)
        {
            // The pixel is on an edge
            if (edge_image.at<unsigned char>(i, j) > 0)
            {
                // Process all the angles
                for (int theta = 0; theta <= 180; ++theta)
                {
                    double angle = deg2rad(theta);
                    double r = i * cos(angle) + j * sin(angle);

                    int v = floor(r + half_height);
                    //cout << theta << " " << v << " " << accumulator.cols << "x" << accumulator.rows << endl;
                    accumulator.at<float>(v, theta) += 1;
                }
            }
        }
    }

    return accumulator;
}

In [11]:
Mat accumulator = houghTransform(image, 60);

## Visualise the accumulator

Look for dots. Every dot represents a line in the original image. There are for of them

In [12]:
Mat vis_accumulator;
cv::normalize(accumulator, vis_accumulator, 0, 1, NORM_MINMAX, CV_32FC1);
    
namedWindow("accumulator", WINDOW_GUI_EXPANDED);
imshow("accumulator", vis_accumulator);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

![accumulator](../wonky_accumulator.png)

## Draw the lines

In [13]:
Mat drawLines(const Mat& anImage,
              const Mat& anAccumulator,
              double aHoughThreshold,
              int aLineWidth,
              const Scalar& aLineColour)
{
    // Copy the input image into the output image
    Mat output = anImage.clone();

    // Process all the pixels of the accumulator image
    for (int j = 0; j < anAccumulator.rows; ++j)
    {
        for (int i = 0; i < anAccumulator.cols; ++i)
        {
            // The pixel value in the accumulator is greater than the threshold
            // Display the corresponding line
            if (anAccumulator.at<float>(j, i) >= aHoughThreshold)
            {
                // The pixel location
                Point location(i, j);

                // The two corners of the image
                Point pt1(               0, 0);
                Point pt2(anImage.cols - 1, anImage.rows - 1);

                // Get theta in radian
                double theta = deg2rad(location.x);

                // Get r
                double r = location.y;
                r -= anAccumulator.rows / 2.0;

                // How to retrieve the line from theta and r:
                //      x = (r - y * sin(theta)) / cos(theta);
                //      y = (r - x * cos(theta)) / sin(theta);

                // sin(theta) != 0
                if (location.x != 0 && location.x != 180)
                {
                    pt1.x = (r - pt1.y * cos(theta)) / sin(theta);
                    pt2.x = (r - pt2.y * cos(theta)) / sin(theta);
                }
                // sin(theta) == 0 && cos(theta) != 0
                else
                {
                    pt1.y = (r - pt1.x * sin(theta)) / cos(theta);
                    pt2.y = (r - pt2.x * sin(theta)) / cos(theta);
                }

                // Draw the line
                line(output, pt1, pt2, aLineColour, aLineWidth);
            }
        }
    }

    return output;
}

In [14]:
// Get tne min and max in the accumulator
double min_value, max_value;
cv::minMaxLoc(accumulator, &min_value, &max_value, 0, 0);

double hough_threshold = min_value + 0.6 * (max_value - min_value);

Mat image_with_lines;
image_with_lines = drawLines(image, accumulator, hough_threshold, 4, Scalar(0, 0, 255));

In [15]:
namedWindow("image", WINDOW_GUI_EXPANDED);
imshow("image", image);

namedWindow("edge", WINDOW_GUI_EXPANDED);
imshow("edge", edge);

namedWindow("image_with_lines", WINDOW_GUI_EXPANDED);
imshow("image_with_lines", image_with_lines);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

| Original image | canny | lines |
|----------------|--------|--------|
|![image](../wonky_original.png) |  ![canny](../wonky_canny.png) | ![ines](../wonky_with_lines.png)

## Extract the angle from the accumulator

In [16]:
double min_value, max_value;
cv::Point min_loc, max_loc;
cv::minMaxLoc(accumulator, &min_value, &max_value, &min_loc, &max_loc);

In [17]:
cout << "Max value: " << max_value << " Location: " << max_loc << endl;

Max value: 418 Location: [9, 1632]


We must convert the position along the horizontal axis into an angle. It's simple. [9, 1632] Tells us that the image is rotated by 9 degrees. To straighten it, we must rotate it by -9 degrees.

In [18]:
Mat rotate(const Mat& anImage, double angle)
{
    Mat rotated_image;
    
    // Point from where to rotate (centre of rotation), here the centre of the image
    Point2f pt(anImage.cols / 2.0, anImage.rows / 2.0);              
    
    // Create a rotation matrix
    Mat rotation_matrix = getRotationMatrix2D(pt, angle, 1.0);
    
    // Apply the transforation to the image
    warpAffine(anImage, rotated_image, rotation_matrix, Size(anImage.cols, anImage.rows));  
    
    return rotated_image;
}

In [19]:
Mat rotated = rotate(image, -max_loc.x);

In [20]:
namedWindow("image", WINDOW_GUI_EXPANDED);
imshow("image", image);

namedWindow("rotated", WINDOW_GUI_EXPANDED);
imshow("rotated", rotated);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

| Original image | straighten |
|----------------|--------|
|![image](../wonky_original.png) | ![straighten](../wonky_straighten.png)